In [ ]:
!pip install datasets nltk sumy evaluate rouge-score

**Frequency Based Method with rouge score**

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from datasets import load_dataset
import evaluate

nltk.download('punkt')
nltk.download('stopwords')

rouge = evaluate.load('rouge')

# Frequency-based summarization function
def frequency_based_summarization(text):
    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)

    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    word_freq = {}
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

    max_freq = max(word_freq.values())
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    sentence_scores = {}
    for sentence in sentences:
        sentence_words = word_tokenize(sentence.lower())
        for word in sentence_words:
            if word in word_freq:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary_sentences = sorted_sentences[:3]

    summary = ' '.join(summary_sentences)
    return summary

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

# Function to summarize multiple articles and calculate ROUGE score
def summarize_cnn_articles(dataset, max_articles=100):
    summaries = []
    reference_summaries = []
    for i in range(min(len(dataset), max_articles)):
        article = dataset[i]['article']
        reference_summary = dataset[i]['highlights']
        summary = frequency_based_summarization(article)
        summaries.append(summary)
        reference_summaries.append(reference_summary)

    # Calculate ROUGE scores
    rouge_scores = rouge.compute(predictions=summaries, references=reference_summaries)

    return summaries, rouge_scores

cnn_summaries, rouge_scores = summarize_cnn_articles(cnn_dataset)

for idx, summary in enumerate(cnn_summaries):
    print(f"Summary of Article {idx + 1}:")
    print(summary)
    print()

print("ROUGE Scores:")
print(rouge_scores)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Summary of Article 1:
"Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014."

Summary of Article 2:
The veterinary hospital's Good Samaritan Fund committee awarded some money to help pay for the dog's treatment, but Mellado has set up a fundraising page to help meet the remaining cost of the dog's care. That's according to Washington State Univers

**Luhn Method Rouge Score**

In [ ]:
# Import necessary libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from datasets import load_dataset
import evaluate

rouge = evaluate.load('rouge')

# Luhn summarization method
def luhn_method(text, sentences_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer_luhn = LuhnSummarizer()

    summary = summarizer_luhn(parser.document, sentences_count)
    summary_sentences = [str(sentence) for sentence in summary]

    return ' '.join(summary_sentences)

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return {key: value * 100 for key, value in rouge_output.items()}

dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

source_texts = dataset['article'][:100]
reference_summaries = dataset['highlights'][:100]

luhn_summaries = [luhn_method(text) for text in source_texts]

rouge_scores = compute_rouge(luhn_summaries, reference_summaries)

print("Luhn Summary:")
print(luhn_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])

print("\nROUGE Scores:")
print(rouge_scores)


Luhn Summary:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement.

Reference Summary:
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

ROUGE Scores:
{'rouge1': 27.16131711742986, 'rouge2': 8.844314520700502, 'rougeL': 18.655483460387483, 'rougeLsum': 21.776711435303753}


**LSA method Rouge Score**

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from datasets import load_dataset
import nltk
import evaluate

nltk.download('punkt')

rouge = evaluate.load('rouge')

# Function for LSA-based summarization
def lsa_summarization(text, sentence_count=2):
    # Create a PlaintextParser and tokenize the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    summarizer = LsaSummarizer()

    summary = summarizer(parser.document, sentence_count)

    summary_sentences = [str(sentence) for sentence in summary]

    final_summary = " ".join(summary_sentences)

    return final_summary

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return {key: value * 100 for key, value in rouge_output.items()}

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

source_texts = cnn_dataset['article'][:100]
reference_summaries = cnn_dataset['highlights'][:100]

lsa_summaries = [lsa_summarization(text, sentence_count=3) for text in source_texts]

rouge_scores = compute_rouge(lsa_summaries, reference_summaries)

print("source Text:")
print(source_texts[0])

print("LSA Summary:")
print(lsa_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])
print("\nROUGE Scores:")
print(rouge_scores)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


source Text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's